**By using the output of this notebook, you are accepting the [competition rules](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/rules).**


## References

- Monochrome fix and scaling: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
- Resizing and saving image: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image

In [ ]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import torchvision.transforms as transforms

import torch
from torch import nn
import glob

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

### get train images

In [ ]:
image_id = []
dim0 = []
dim1 = []
resized_train={}

#add normalize also
my_transform = transforms.Compose([
    transforms.ToTensor(),
])
        
load_train_dir = f'../input/vinbigdata-chest-xray-abnormalities-detection/train/*'
save_train_dir = "./train/"

train_data = glob.glob(load_train_dir)
print("total labelled data count : {}".format(len(train_data)))
print("sample path : {}".format(train_data[0]))


os.makedirs(save_train_dir, exist_ok=True)
    
for file in train_data[0:20]:

    # set keep_ratio=True to have original aspect ratio
    xray = read_xray(file)
    im = resize(xray, size=512) 

    img_id = file.split("/train/")[1].split(".")
    img_id = img_id[0]
    print(img_id)
    im.save(save_train_dir+img_id+'.png')
    print(save_train_dir+img_id+'.png')


    resized_train[img_id] = my_transform(im)

    image_id.append(image_id)
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])


### get test images

In [ ]:
test_image_id = []
test_dim0 = []
test_dim1 = []
resized_test={}


#add normalize also
my_test_transform = transforms.Compose([
    transforms.ToTensor(),
])
 
    
load_test_dir = f'../input/vinbigdata-chest-xray-abnormalities-detection/train/*'
save_test_dir = f'./test/'

# get test data path
test_data = glob.glob(load_test_dir)


os.makedirs(save_test_dir, exist_ok=True)
    
# get last 1000 or 10 ( just for testing )

for file in test_data[-10:]:
    
#     print("file {}".format(file))

    # set keep_ratio=True to have original aspect ratio
    xray = read_xray(file)
    im = resize(xray, size=512) 

    test_img_id = file.split("/train/")[1].split(".")
    test_img_id = test_img_id[0]
#     print(test_img_id)

    im.save(save_test_dir + test_img_id + ".png")
    
    resized_test[test_img_id] = my_transform(im)

    test_image_id.append(test_img_id)
    test_dim0.append(xray.shape[0])
    test_dim1.append(xray.shape[1])

    


### test train count

In [ ]:
print("train images count : {}".format(len(resized_train)))

print("test images count : {}".format(len(resized_test)))
    
print(resized_train.keys())

### save train n test data in zip ( can be downloaded from output directory -> )

In [ ]:
%%time

#save train resized images in zip
!tar -zcf train.tar.gz -C "./train/" .


#save test resized images in zip
!tar -zcf train.tar.gz -C "./test/" .


In [ ]:
# df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1})
# df.to_csv('train_meta.csv', index=False)



In [ ]:
# df.head()


In [ ]:
# df.shape

In [ ]:
#image datatype
# resized_train

In [ ]:
k = list(resized_train.keys())
k

In [ ]:
from IPython.display import display
k = list(resized_train.keys())
display(resized_train[k[1]])
k

In [ ]:
# show imahge

import matplotlib.pyplot as plt

img = resized_train[list(resized_train.keys())[1]]
plt.imshow(img.squeeze(), cmap=plt.cm.gray)
plt.show()


print(img.shape)
img_unsq = img.unsqueeze(0)
print(img_unsq.shape)


### get train labels ( just classes for now)

### Will use fast rcnn for detection i,e bounding box

In [ ]:
labels = pd.read_csv("/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
print("Train Data Size : {}".format(labels.shape[0]))
labels.head()

In [ ]:
class_labels = labels.iloc[:,[0,2]]
class_labels.head()

In [ ]:
class_labels.iloc[:,0:2]
class_labels = class_labels.drop_duplicates(subset=["image_id"])
class_labels.head()
class_labels.shape

### model : training, testing

In [ ]:
train_on_gpu = False

if torch.cuda.is_available():
    train_on_gpu = True

In [ ]:
# model

import torch
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 512x512x3 image tensor)
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1)

        # convolutional layer (sees 256x256x4 tensor)
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1)
        
        # convolutional layer (sees 128x128x8 tensor)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1)
        
        # convolutional layer (sees 64x64x16 tensor)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1)
        
        # convolutional layer (sees 32x32x32  tensor)
        self.conv5 = nn.Conv2d(32, 64, 3, padding=1)
    
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
    
        self.fc1 = nn.Linear(64 * 16 * 16, 1600)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(1600, 512)
        # dropout layer (p=0.25)
        self.fc3 = nn.Linear(512,15)

        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        # flatten image input

        x = x.view(-1, 64 * 16 * 16)
        # add dropout layer
        
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = F.relu(self.fc2(x))

        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc3(x)
        
        return x

# create a complete CNN
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if torch.cuda.is_available():
    model.cuda()

In [ ]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:

# number of epochs to train the model
n_epochs = 2

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):
    print("********* FOR EPOCH : {} ****************".format(epoch))
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data_id,data in resized_train.items():
        # move tensors to GPU if CUDA is available
        target = torch.tensor(int(class_labels.loc[class_labels["image_id"]==data_id]["class_id"]))
        target = target.unsqueeze(0)
        data = data.unsqueeze(0)
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
#         output = torch.argmax(output)
#         print("output : ",output.shape)
#         print("target : ",target.shape)
#         print("target item : ", target.item())
        
        loss = criterion(output, target)
        print("H",loss)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
#     model.eval()
#     for data, target in valid_loader:
#         # move tensors to GPU if CUDA is available
#         if train_on_gpu:
#             data, target = data.cuda(), target.cuda()
#         # forward pass: compute predicted outputs by passing inputs to the model
#         output = model(data)
#         # calculate the batch loss
#         loss = criterion(output, target)
#         # update average validation loss 
#         valid_loss += loss.item()*data.size(0)
    
#     # calculate average losses
#     train_loss = train_loss/len(train_loader.sampler)
#     valid_loss = valid_loss/len(valid_loader.sampler)
        
#     # print training/validation statistics 
#     print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
#         epoch, train_loss, valid_loss))
    
#     # save model if validation loss has decreased
#     if valid_loss <= valid_loss_min:
#         print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
#         valid_loss_min,
#         valid_loss))
#         torch.save(model.state_dict(), 'model_cifar.pt')
#         valid_loss_min = valid_loss

### test


In [ ]:
# track test loss
batch_size = 1
test_loss = 0.0
class_correct = list(0. for i in range(15))
class_total = list(0. for i in range(15))

model.eval()
# iterate over test data
for data_id,data in resized_test.items():

    target = torch.tensor(int(class_labels.loc[class_labels["image_id"]==data_id]["class_id"]))
    target = target.unsqueeze(0)
    data = data.unsqueeze(0)
    
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    
    # update test loss 
    test_loss += loss.item()*data.size(0)
    
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1) ## returns max value as frst, and index of max value as second   
    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    
    print("\n",correct_tensor)
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        print("predicted label {}".format(pred.item()))
        print("actual label {}".format(target.item()))
        
#         print(correct)
#         print(correct[i].item())
        class_correct[label] += correct
        class_total[label] += 1

# average test loss
# test_loss = test_loss/len(test_loader.dataset)
test_loss = test_loss/10
print('Test Loss: {:.6f}\n'.format(test_loss))

classes = range(0,15)
for i in range(15):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))